# Y2018M02D27_RH_Moving_Average_Demand_EE_V01

* Purpose of script: Moving average for demand at basin resolution. The script will calculate the volumetric and flux 10 year moving average at a Pfaf6 basin level for total demand. (potentially also per sector demand)

* Script exports to: 
* projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PTotWW_month_m_pfaf06_1960_2014_movingaverage_10y_V01
* projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PTotWN_month_m_pfaf06_1960_2014_movingaverage_10y_V01
* Kernel used: python35
* Date created: 20170227

In [1]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M02D28 UTC 12:35


'3.5.4 |Continuum Analytics, Inc.| (default, Aug 14 2017, 13:26:58) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [2]:
SCRIPT_NAME = "Y2018M02D27_RH_Moving_Average_Demand_EE_V01"

CRS = "EPSG:4326"

EE_PATH = "projects/WRI-Aquaduct/PCRGlobWB20V07"

OUTPUT_VERSION = 3

DIMENSION30S = {}
DIMENSION30S["x"] = 43200
DIMENSION30S["y"] = 21600


MA_WINDOW_LENGTH = 10 # Moving average window length. 

TESTING = 0

In [3]:
import ee
import os
import logging
import pandas as pd
import subprocess

In [4]:
ee.Initialize()

In [5]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
file_handler = logging.FileHandler("./logs/%sV%0.2d.log" %(SCRIPT_NAME,OUTPUT_VERSION))
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [6]:
geometry = ee.Geometry.Polygon(coords=[[-180.0, -90.0], [180,  -90.0], [180, 90], [-180,90]], proj= ee.Projection('EPSG:4326'),geodesic=False )


In [7]:
crsTransform30sSmall = [
    360.0 / DIMENSION30S["x"], 
    0,
    -180,
    0,
    -162.0 / (0.9* DIMENSION30S["y"]),
    81   
]

dimensions30sSmall = "{}x{}".format(DIMENSION30S["x"],int(0.9*DIMENSION30S["y"]))

The images in the imagecollection have two bands: count and mean. 

In [8]:
def create_collection(assetid):
    """ Create image collection in earth engine asset folder
    
    This function will only work if the folder in which the
    new imageCollection will be created is valid
    
    
    Args:
        assetid (string) : asset id for the new image collection
    
    Returns: 
        result (string) : captured message from command line
    
    """   
    
    command = "earthengine create collection {}".format(assetid) 
    result = subprocess.check_output(command,shell=True)
    if result:
        logger.error(result)
    return result 

def moving_average_decade(year,ic):
    """ Calculate a 10 year moving average
    
    This function is limited to one input paramater to allow mapping over a simple list. 
    Averages the 10 year up to the input year. (]
    
    Global variables required include an imageCollection with a year property.
    
    
    Args:
        year (integer) : final year of interest.
        ic (ee.ImageCollection) : input imageCollection
    
    Returns: 
        image (ee.Image) : earth engine image with the mean of the last 10 years
    """
    
    min_year = year - MA_WINDOW_LENGTH
    
    ic_filtered = (ic.filter(ee.Filter.gt("year",min_year))
                     .filter(ee.Filter.lte("year",year))
                  )
    
    i_mean = ic_filtered.reduce(ee.Reducer.mean()) 
    
    
    i_mean = i_mean.copyProperties(source=ic_filtered.first(),
                          exclude=["script_used",
                                   "output_version",
                                   "year",
                                   "output_version",
                                   "version",
                                   "reducer"])
    
    return ee.Image(i_mean)


def set_properties(image):
    """ Set properties to image based on rows in pandas dataframe
    
    Args:
        image (ee.Image) : image without properties
        
    Returns:
        image_out (ee.Image) : image with properties
    """
    
    properties ={}
    properties["year"] = row["year"]
    properties["month"] = row["month"]
    properties["moving_average_length"] = MA_WINDOW_LENGTH
    properties["moving_average_year_min"] = row["year"]- (MA_WINDOW_LENGTH-1)
    properties["script_used"] = SCRIPT_NAME
    properties["indicator"] = row["indicator"]
    properties["version"] = OUTPUT_VERSION
    properties["spatial_resolution"] = "30s"
    properties["exportdescription"] = row["exportdescription"]
    
    image_out = ee.Image(image).set(properties)
    return image_out


def export_asset(image):
    """ Export a google earth engine image to an asset folder
    
    function will start a new task. To view the status of the task
    check the javascript API or query tasks script. Function is used 
    as mapped function so other arguments need to be set globally. 
    
    Args:
        image (ee.Image) : Image to export
        
    Returns:
        asset_id (string) : asset id of     
    """
    
    asset_id = row["output_i_assetid"]
    task = ee.batch.Export.image.toAsset(
        image =  ee.Image(image),
        description = "{}V{}".format(row["exportdescription"],OUTPUT_VERSION),
        assetId = asset_id,
        dimensions = dimensions30sSmall,
        crs = CRS,
        crsTransform = crsTransform30sSmall,
        maxPixels = 1e10     
    )
    task.start()
    return asset_id



    

In [9]:
months = range(1,13)
years = range(1960+9,2014+1)
indicators = ["PTotWW","PTotWN"]

It is only possible to calculate a 10 year running mean starting in 1969

In [10]:
df = pd.DataFrame()

for indicator in indicators:
    for month in months:
        for year in years:
            newRow = {}
            newRow["month"] = month
            newRow["year"] = year
            newRow["output_ic_filename"] = "global_historical_{}_month_m_pfaf06_1960_2014_movingaverage_10y_V{:02.0f}".format(indicator,OUTPUT_VERSION)
            newRow["output_ic_assetid"] = "{}/{}".format(EE_PATH,newRow["output_ic_filename"])
            newRow["output_i_filename"] = "global_historical_{}_month_m_pfaf06_Y{:04.0f}M{:02.0f}_movingaverage_10y_V{:02.0f}".format(indicator,year,month,OUTPUT_VERSION)
            newRow["output_i_assetid"] = "{}/{}".format(newRow["output_ic_assetid"],newRow["output_i_filename"])
            newRow["indicator"] = indicator
            newRow["exportdescription"] = "{}_month_Y{:04.0f}M{:02.0f}_movingaverage_10y".format(indicator,year,month)
            df= df.append(newRow,ignore_index=True)


In [11]:
df.head()

,exportdescription,indicator,month,output_i_assetid,output_i_filename,output_ic_assetid,output_ic_filename,year
0,PTotWW_month_Y1969M01_movingaverage_10y,PTotWW,1.0,projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...,global_historical_PTotWW_month_m_pfaf06_Y1969M...,projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...,global_historical_PTotWW_month_m_pfaf06_1960_2...,1969.0
1,PTotWW_month_Y1970M01_movingaverage_10y,PTotWW,1.0,projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...,global_historical_PTotWW_month_m_pfaf06_Y1970M...,projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...,global_historical_PTotWW_month_m_pfaf06_1960_2...,1970.0
2,PTotWW_month_Y1971M01_movingaverage_10y,PTotWW,1.0,projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...,global_historical_PTotWW_month_m_pfaf06_Y1971M...,projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...,global_historical_PTotWW_month_m_pfaf06_1960_2...,1971.0
3,PTotWW_month_Y1972M01_movingaverage_10y,PTotWW,1.0,projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...,global_historical_PTotWW_month_m_pfaf06_Y1972M...,projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...,global_historical_PTotWW_month_m_pfaf06_1960_2...,1972.0
4,PTotWW_month_Y1973M01_movingaverage_10y,PTotWW,1.0,projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...,global_historical_PTotWW_month_m_pfaf06_Y1973M...,projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...,global_historical_PTotWW_month_m_pfaf06_1960_2...,1973.0


In [12]:
if TESTING:
    df = df[0:2]

In [13]:
df.shape

(1104, 8)

In [14]:
df.loc[0]

exportdescription               PTotWW_month_Y1969M01_movingaverage_10y
indicator                                                        PTotWW
month                                                                 1
output_i_assetid      projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...
output_i_filename     global_historical_PTotWW_month_m_pfaf06_Y1969M...
output_ic_assetid     projects/WRI-Aquaduct/PCRGlobWB20V07/global_hi...
output_ic_filename    global_historical_PTotWW_month_m_pfaf06_1960_2...
year                                                               1969
Name: 0, dtype: object

In [15]:
for output_ic_assetid in df["output_ic_assetid"].unique():
    result = create_collection(output_ic_assetid)
    print(result)

b'Asset projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PTotWW_month_m_pfaf06_1960_2014_movingaverage_10y_V03 already exists\n'
b''


In [16]:
function_time_start = datetime.datetime.now()
for index, row in df.iterrows():    

    ic = ee.ImageCollection("{}/global_historical_{}_month_m_pfaf06_1960_2014".format(EE_PATH,row["indicator"]))
    ic = ic.select(["mean"])
    ic_month = ic.filter(ee.Filter.eq("month",row["month"]))
    print(ic_month.first().get("month").getInfo())
    i_mean = moving_average_decade(row["year"],ic_month)
    i_mean = set_properties(i_mean)
    asset_id = export_asset(i_mean)
    logger.info(asset_id)
    elapsed = datetime.datetime.now() - function_time_start
    print("Processing image {} month {} of year {} runtime {}".format(index,row["month"],row["year"],elapsed))

1.0
Processing image 0 month 1.0 of year 1969.0 runtime 0:00:02.585201
1.0
Processing image 1 month 1.0 of year 1970.0 runtime 0:00:05.044180
1.0
Processing image 2 month 1.0 of year 1971.0 runtime 0:00:07.515344
1.0
Processing image 3 month 1.0 of year 1972.0 runtime 0:00:10.062479
1.0
Processing image 4 month 1.0 of year 1973.0 runtime 0:00:12.500944
1.0
Processing image 5 month 1.0 of year 1974.0 runtime 0:00:15.126928
1.0
Processing image 6 month 1.0 of year 1975.0 runtime 0:00:17.496326
1.0
Processing image 7 month 1.0 of year 1976.0 runtime 0:00:19.979097
1.0
Processing image 8 month 1.0 of year 1977.0 runtime 0:00:22.434428
1.0
Processing image 9 month 1.0 of year 1978.0 runtime 0:00:24.395661
1.0
Processing image 10 month 1.0 of year 1979.0 runtime 0:00:26.836922
1.0
Processing image 11 month 1.0 of year 1980.0 runtime 0:00:29.291972
1.0
Processing image 12 month 1.0 of year 1981.0 runtime 0:00:31.703306
1.0
Processing image 13 month 1.0 of year 1982.0 runtime 0:00:34.103029
1.

3.0
Processing image 114 month 3.0 of year 1991.0 runtime 0:04:28.731876
3.0
Processing image 115 month 3.0 of year 1992.0 runtime 0:04:31.106225
3.0
Processing image 116 month 3.0 of year 1993.0 runtime 0:04:33.575195
3.0
Processing image 117 month 3.0 of year 1994.0 runtime 0:04:35.992882
3.0
Processing image 118 month 3.0 of year 1995.0 runtime 0:04:38.498959
3.0
Processing image 119 month 3.0 of year 1996.0 runtime 0:04:40.949588
3.0
Processing image 120 month 3.0 of year 1997.0 runtime 0:04:43.372737
3.0
Processing image 121 month 3.0 of year 1998.0 runtime 0:04:45.793947
3.0
Processing image 122 month 3.0 of year 1999.0 runtime 0:04:48.270063
3.0
Processing image 123 month 3.0 of year 2000.0 runtime 0:04:50.599182
3.0
Processing image 124 month 3.0 of year 2001.0 runtime 0:04:52.335328
3.0
Processing image 125 month 3.0 of year 2002.0 runtime 0:04:54.607413
3.0
Processing image 126 month 3.0 of year 2003.0 runtime 0:04:57.101643
3.0
Processing image 127 month 3.0 of year 2004.0 r

5.0
Processing image 227 month 5.0 of year 2012.0 runtime 0:08:41.962342
5.0
Processing image 228 month 5.0 of year 2013.0 runtime 0:08:44.340621
5.0
Processing image 229 month 5.0 of year 2014.0 runtime 0:08:46.426505
6.0
Processing image 230 month 6.0 of year 1969.0 runtime 0:08:48.815658
6.0
Processing image 231 month 6.0 of year 1970.0 runtime 0:08:50.883310
6.0
Processing image 232 month 6.0 of year 1971.0 runtime 0:08:53.274302
6.0
Processing image 233 month 6.0 of year 1972.0 runtime 0:08:55.368770
6.0
Processing image 234 month 6.0 of year 1973.0 runtime 0:08:57.763585
6.0
Processing image 235 month 6.0 of year 1974.0 runtime 0:08:59.839211
6.0
Processing image 236 month 6.0 of year 1975.0 runtime 0:09:01.751329
6.0
Processing image 237 month 6.0 of year 1976.0 runtime 0:09:04.143674
6.0
Processing image 238 month 6.0 of year 1977.0 runtime 0:09:06.528384
6.0
Processing image 239 month 6.0 of year 1978.0 runtime 0:09:08.971869
6.0
Processing image 240 month 6.0 of year 1979.0 r

8.0
Processing image 340 month 8.0 of year 1987.0 runtime 0:12:52.706492
8.0
Processing image 341 month 8.0 of year 1988.0 runtime 0:12:55.142353
8.0
Processing image 342 month 8.0 of year 1989.0 runtime 0:12:57.105476
8.0
Processing image 343 month 8.0 of year 1990.0 runtime 0:13:00.374887
8.0
Processing image 344 month 8.0 of year 1991.0 runtime 0:13:01.860764
8.0
Processing image 345 month 8.0 of year 1992.0 runtime 0:13:04.572310
8.0
Processing image 346 month 8.0 of year 1993.0 runtime 0:13:06.507408
8.0
Processing image 347 month 8.0 of year 1994.0 runtime 0:13:09.213312
8.0
Processing image 348 month 8.0 of year 1995.0 runtime 0:13:11.590663
8.0
Processing image 349 month 8.0 of year 1996.0 runtime 0:13:13.987793
8.0
Processing image 350 month 8.0 of year 1997.0 runtime 0:13:15.472342
8.0
Processing image 351 month 8.0 of year 1998.0 runtime 0:13:17.917141
8.0
Processing image 352 month 8.0 of year 1999.0 runtime 0:13:20.875364
8.0
Processing image 353 month 8.0 of year 2000.0 r

10.0
Processing image 452 month 10.0 of year 2007.0 runtime 0:17:03.285541
10.0
Processing image 453 month 10.0 of year 2008.0 runtime 0:17:05.512443
10.0
Processing image 454 month 10.0 of year 2009.0 runtime 0:17:07.759218
10.0
Processing image 455 month 10.0 of year 2010.0 runtime 0:17:10.054642
10.0
Processing image 456 month 10.0 of year 2011.0 runtime 0:17:12.033282
10.0
Processing image 457 month 10.0 of year 2012.0 runtime 0:17:14.407925
10.0
Processing image 458 month 10.0 of year 2013.0 runtime 0:17:16.418244
10.0
Processing image 459 month 10.0 of year 2014.0 runtime 0:17:18.444603
11.0
Processing image 460 month 11.0 of year 1969.0 runtime 0:17:19.962915
11.0
Processing image 461 month 11.0 of year 1970.0 runtime 0:17:22.344051
11.0
Processing image 462 month 11.0 of year 1971.0 runtime 0:17:24.782112
11.0
Processing image 463 month 11.0 of year 1972.0 runtime 0:17:27.171113
11.0
Processing image 464 month 11.0 of year 1973.0 runtime 0:17:29.154559
11.0
Processing image 465

1.0
Processing image 562 month 1.0 of year 1979.0 runtime 0:20:57.965027
1.0
Processing image 563 month 1.0 of year 1980.0 runtime 0:21:00.383767
1.0
Processing image 564 month 1.0 of year 1981.0 runtime 0:21:02.679202
1.0
Processing image 565 month 1.0 of year 1982.0 runtime 0:21:05.097280
1.0
Processing image 566 month 1.0 of year 1983.0 runtime 0:21:07.519244
1.0
Processing image 567 month 1.0 of year 1984.0 runtime 0:21:09.445579
1.0
Processing image 568 month 1.0 of year 1985.0 runtime 0:21:11.890572
1.0
Processing image 569 month 1.0 of year 1986.0 runtime 0:21:13.950371
1.0
Processing image 570 month 1.0 of year 1987.0 runtime 0:21:15.884149
1.0
Processing image 571 month 1.0 of year 1988.0 runtime 0:21:17.852732
1.0
Processing image 572 month 1.0 of year 1989.0 runtime 0:21:19.878925
1.0
Processing image 573 month 1.0 of year 1990.0 runtime 0:21:21.961057
1.0
Processing image 574 month 1.0 of year 1991.0 runtime 0:21:23.944324
1.0
Processing image 575 month 1.0 of year 1992.0 r

3.0
Processing image 675 month 3.0 of year 2000.0 runtime 0:24:55.999222
3.0
Processing image 676 month 3.0 of year 2001.0 runtime 0:24:58.425420
3.0
Processing image 677 month 3.0 of year 2002.0 runtime 0:25:00.456407
3.0
Processing image 678 month 3.0 of year 2003.0 runtime 0:25:02.842372
3.0
Processing image 679 month 3.0 of year 2004.0 runtime 0:25:05.279283
3.0
Processing image 680 month 3.0 of year 2005.0 runtime 0:25:07.298990
3.0
Processing image 681 month 3.0 of year 2006.0 runtime 0:25:09.253713
3.0
Processing image 682 month 3.0 of year 2007.0 runtime 0:25:10.790985
3.0
Processing image 683 month 3.0 of year 2008.0 runtime 0:25:12.720664
3.0
Processing image 684 month 3.0 of year 2009.0 runtime 0:25:14.679936
3.0
Processing image 685 month 3.0 of year 2010.0 runtime 0:25:16.147219
3.0
Processing image 686 month 3.0 of year 2011.0 runtime 0:25:18.621040
3.0
Processing image 687 month 3.0 of year 2012.0 runtime 0:25:20.608399
3.0
Processing image 688 month 3.0 of year 2013.0 r

6.0
Processing image 788 month 6.0 of year 1975.0 runtime 0:29:02.055722
6.0
Processing image 789 month 6.0 of year 1976.0 runtime 0:29:04.545848
6.0
Processing image 790 month 6.0 of year 1977.0 runtime 0:29:06.990543
6.0
Processing image 791 month 6.0 of year 1978.0 runtime 0:29:09.018262
6.0
Processing image 792 month 6.0 of year 1979.0 runtime 0:29:11.396057
6.0
Processing image 793 month 6.0 of year 1980.0 runtime 0:29:13.844477
6.0
Processing image 794 month 6.0 of year 1981.0 runtime 0:29:15.758081
6.0
Processing image 795 month 6.0 of year 1982.0 runtime 0:29:17.703386
6.0
Processing image 796 month 6.0 of year 1983.0 runtime 0:29:19.702600
6.0
Processing image 797 month 6.0 of year 1984.0 runtime 0:29:22.185896
6.0
Processing image 798 month 6.0 of year 1985.0 runtime 0:29:24.578701
6.0
Processing image 799 month 6.0 of year 1986.0 runtime 0:29:26.975051
6.0
Processing image 800 month 6.0 of year 1987.0 runtime 0:29:29.333612
6.0
Processing image 801 month 6.0 of year 1988.0 r

Processing image 901 month 8.0 of year 1996.0 runtime 0:33:13.177373
8.0
Processing image 902 month 8.0 of year 1997.0 runtime 0:33:15.440225
8.0
Processing image 903 month 8.0 of year 1998.0 runtime 0:33:16.737866
8.0
Processing image 904 month 8.0 of year 1999.0 runtime 0:33:18.439694
8.0
Processing image 905 month 8.0 of year 2000.0 runtime 0:33:20.680708
8.0
Processing image 906 month 8.0 of year 2001.0 runtime 0:33:22.874180
8.0
Processing image 907 month 8.0 of year 2002.0 runtime 0:33:25.263911
8.0
Processing image 908 month 8.0 of year 2003.0 runtime 0:33:27.001721
8.0
Processing image 909 month 8.0 of year 2004.0 runtime 0:33:28.798974
8.0
Processing image 910 month 8.0 of year 2005.0 runtime 0:33:31.009197
8.0
Processing image 911 month 8.0 of year 2006.0 runtime 0:33:33.502726
8.0
Processing image 912 month 8.0 of year 2007.0 runtime 0:33:35.716554
8.0
Processing image 913 month 8.0 of year 2008.0 runtime 0:33:37.921041
8.0
Processing image 914 month 8.0 of year 2009.0 runti

11.0
Processing image 1012 month 11.0 of year 1969.0 runtime 0:37:19.464662
11.0
Processing image 1013 month 11.0 of year 1970.0 runtime 0:37:21.634856
11.0
Processing image 1014 month 11.0 of year 1971.0 runtime 0:37:23.884279
11.0
Processing image 1015 month 11.0 of year 1972.0 runtime 0:37:26.068486
11.0
Processing image 1016 month 11.0 of year 1973.0 runtime 0:37:28.243210
11.0
Processing image 1017 month 11.0 of year 1974.0 runtime 0:37:30.450835
11.0
Processing image 1018 month 11.0 of year 1975.0 runtime 0:37:32.181541
11.0
Processing image 1019 month 11.0 of year 1976.0 runtime 0:37:34.379739
11.0
Processing image 1020 month 11.0 of year 1977.0 runtime 0:37:36.583907
11.0
Processing image 1021 month 11.0 of year 1978.0 runtime 0:37:38.308609
11.0
Processing image 1022 month 11.0 of year 1979.0 runtime 0:37:40.529830
11.0
Processing image 1023 month 11.0 of year 1980.0 runtime 0:37:42.282396
11.0
Processing image 1024 month 11.0 of year 1981.0 runtime 0:37:44.115159
11.0
Process

In [17]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:40:45.186689
